In [1]:
from pulp import *

In [9]:
NODES = ['N1','N2','N3','N4','N5', 'N6']

LINKS = [('N1','N2'), ('N1','N6'), ('N2','N3'), ('N2','N5'), ('N3','N5'), ('N3','N4'), ('N4','N5'), ('N6','N5')]

F = [1,2,3,4,5]

demands = {
    1 : ('N1', 'N5', 3),
    2 : ('N1', 'N4', 2),
    3 : ('N1', 'N3', 1),
    4 : ('N1', 'N6', 2)
}

paths_demands = {
    1 : [
        frozenset({('N1','N2'),('N2','N5')}),
        frozenset({('N1','N6'),('N6','N5')}),
    ],
    2 : [
        frozenset({('N1','N2'),('N2','N3'), ('N3','N4')}),
    ],
    3 : [
        frozenset({('N1','N2'),('N2','N3')}),
    ],
    4 : [
        frozenset({('N1','N6')}),
    ]
}

paths = [
    frozenset({('N1','N2'),('N2','N5')}),
    frozenset({('N1','N6'),('N6','N5')}),
    frozenset({('N1','N2'),('N2','N3'), ('N3','N4')}),
    frozenset({('N1','N2'),('N2','N3')}),
    frozenset({('N1','N6')})
]

pe = {
    ('N1','N2') : [ 
                    frozenset({('N1','N2'),('N2','N5')}),
                    frozenset({('N1','N2'),('N2','N3'), ('N3','N4')}),
                    frozenset({('N1','N2'),('N2','N3')})
                  ],
    ('N2','N3') : [ 
                    frozenset({('N1','N2'),('N2','N3'), ('N3','N4')}),
                    frozenset({('N1','N2'),('N2','N3')})
                  ],
    ('N3','N4') : [ 
                    frozenset({('N1','N2'),('N2','N3'), ('N3','N4')})
                  ],
    ('N2','N5') : [ 
                    frozenset({('N1','N2'),('N2','N5')})
                    
                  ],
    ('N3','N5') : [ ],
    ('N4','N5') : [ ],
    ('N1','N6') : [ 
                    frozenset({('N1','N6')}),
                    frozenset({('N1','N6'),('N6','N5')})
                  ],
    ('N6','N5') : [ 
                    frozenset({('N1','N6'),('N6','N5')})
                  ]
}

In [15]:



#problemvariables
prob = LpProblem("Problem", LpMinimize)

x1=LpVariable("F",0,None,LpInteger)

Xpf_vars = LpVariable.dicts("Xpf", [(p,f) for p in paths for f in F], 0, 1, LpBinary)

Ypf_vars = LpVariable.dicts("Ypf", [(p,f) for p in paths for f in F], 0, 1, LpBinary)

Xef_vars = LpVariable.dicts("Xef", [(e,f) for e in LINKS for f in F], 0, 1, LpBinary)

Xf_vars = LpVariable.dicts("Xf", F, 0, 1, LpBinary)







In [16]:





prob += lpSum(Xef_vars)


for d in demands:
    for p in paths_demands:
        if(d == p):
            prob += lpSum(Xpf_vars[pd,f] for pd in paths_demands[d] for f in F) == 1


for d in demands:
    for p in paths_demands:
        if(d == p):
            for pd in paths_demands[d]:
                for i in F:
                    if(i <= len(F)-demands[d][2]+1):
                        for j in range(i,len(F)+1): 
                            if(j <= i + demands[d][2] -1):
                                prob += Xpf_vars[pd,i] - Ypf_vars[pd,j] <= 0

                                
for d in demands:
    for p in paths_demands:
        if(d == p):
            for pd in paths_demands[d]:
                for i in F:
                    if( i >= len(F)-demands[d][2]+2):
                        prob += Xpf_vars[pd,i] == 0
                        


for p in pe:
    for e in LINKS:
        if(p == e):
            for f in F:
                 prob += lpSum(Ypf_vars[pp,f] for pp in pe[p]) - Xef_vars[e,f] == 0


    
for f in F:
    prob += lpSum(Xef_vars[e,f] for e in LINKS) - len(F) * Xf_vars[f] <= 0

    
prob += lpSum(Xf_vars[f] for f in F) - x1 == 0








In [17]:
prob.solve()

1

In [20]:
%%time

print('\nMinimum Profit:')
pulp.value(prob.objective)


Minimum Profit:
CPU times: user 332 µs, sys: 121 µs, total: 453 µs
Wall time: 424 µs


16.0

In [19]:
# Each of the variables is printed with it's resolved optimum value
for v in prob.variables():
    print(v.name, "=", v.varValue)

F = 5.0
Xef_(('N1',_'N2'),_1) = 1.0
Xef_(('N1',_'N2'),_2) = 0.0
Xef_(('N1',_'N2'),_3) = 1.0
Xef_(('N1',_'N2'),_4) = 1.0
Xef_(('N1',_'N2'),_5) = 0.0
Xef_(('N1',_'N6'),_1) = 1.0
Xef_(('N1',_'N6'),_2) = 1.0
Xef_(('N1',_'N6'),_3) = 1.0
Xef_(('N1',_'N6'),_4) = 1.0
Xef_(('N1',_'N6'),_5) = 1.0
Xef_(('N2',_'N3'),_1) = 1.0
Xef_(('N2',_'N3'),_2) = 0.0
Xef_(('N2',_'N3'),_3) = 1.0
Xef_(('N2',_'N3'),_4) = 1.0
Xef_(('N2',_'N3'),_5) = 0.0
Xef_(('N2',_'N5'),_1) = 0.0
Xef_(('N2',_'N5'),_2) = 0.0
Xef_(('N2',_'N5'),_3) = 0.0
Xef_(('N2',_'N5'),_4) = 0.0
Xef_(('N2',_'N5'),_5) = 0.0
Xef_(('N3',_'N4'),_1) = 0.0
Xef_(('N3',_'N4'),_2) = 0.0
Xef_(('N3',_'N4'),_3) = 1.0
Xef_(('N3',_'N4'),_4) = 1.0
Xef_(('N3',_'N4'),_5) = 0.0
Xef_(('N3',_'N5'),_1) = 0.0
Xef_(('N3',_'N5'),_2) = 0.0
Xef_(('N3',_'N5'),_3) = 0.0
Xef_(('N3',_'N5'),_4) = 0.0
Xef_(('N3',_'N5'),_5) = 0.0
Xef_(('N4',_'N5'),_1) = 0.0
Xef_(('N4',_'N5'),_2) = 0.0
Xef_(('N4',_'N5'),_3) = 0.0
Xef_(('N4',_'N5'),_4) = 0.0
Xef_(('N4',_'N5'),_5) = 0.0
Xef_(('N6',_